In [1]:
import sprit
import obspy
import pandas as pd

def batch_data_read(input_data, batch_type='csv', param_col=None, batch_params=None, verbose=False, **readcsv_getMeta_fetch_kwargs):
    """Function to read list of files and parameters

    Parameters
    ----------
    input_data : filepath or list
        Filepath to file that can be read using pandas.read_csv()
    batch_type : str {'csv', 'list'}

    Returns
    -------
    stream_dict : dict
        Dictionary contiaining unique streams
    """

    # Dictionary to store the stream objects
    stream_dict = {}
    data_dict = {}
    if batch_type == 'csv':
        #Read csv
        read_csv_kwargs = {k: v for k, v in locals()['readcsv_getMeta_fetch_kwargs'].items() if k in pd.read_csv.__code__.co_varnames}
        dataReadInfoDF = pd.read_csv(input_data, **read_csv_kwargs, verbose=verbose)
        #***WHAT ABOUT THE NANS? nan ***
        
        print(dataReadInfoDF)
        #First figure out columns
        input_params_params = sprit.input_params.__code__.co_varnames
        get_metadata_params = sprit.get_metadata.__code__.co_varnames
        fetch_data_params = sprit.fetch_data.__code__.co_varnames

        param_dict_list = []
        if param_col is None: #Not a single parameter column, each col=parameter
            for row_ind in range(dataReadInfoDF.shape[0]):
                param_dict = {}
                for col in dataReadInfoDF.columns:
                    if col in input_params_params or col in get_metadata_params or col in fetch_data_params:
                        param_dict[col] = dataReadInfoDF.loc[row_ind, col]
                param_dict_list.append(param_dict)
        else:
            for row in dataReadInfoDF[param_col]:
                param_dict = {}
                splitRow = str(row).split(',')
                for item in splitRow:
                    param_dict[item.split('=')[0]] = item.split('=')[1]
                param_dict_list.append(param_dict)
        #input_params(datapath,site,network,station,loc,channels, acq_date,starttime, endtime, tzone, xcoord, ycoord, elevation, depth, instrument, metapath, hvsr_band)
        #fetch_data(params, inv, source, trim_dir, export_format, detrend, detrend_order, verbose)
        #get_metadata(params, write_path)

        #file_setup_kwargs = {k: v for k, v in locals()['keyword_parameters'].items() if k in w4h.file_setup.__code__.co_varnames}
        print(param_dict_list)
    elif batch_type == 'filelist':
        # Read and process each MiniSEED file
        for i, file in enumerate(input_data):
            if batch_params is None:
                pass
            elif isinstance(batch_params, list):
                read_params = batch_params[i]
            elif isinstance(batch_params, dict):
                pass
                #Update this eventually

    hvsr_metaDict = {}
    for param_dict in param_dict_list:
        # Read the data file into a Stream object
        input_params_kwargs = {k: v for k, v in locals()['readcsv_getMeta_fetch_kwargs'].items() if k in sprit.input_params.__code__.co_varnames}
        input_params_kwargs2 = {k: v for k, v in param_dict.items() if k in sprit.input_params.__code__.co_varnames}
        input_params_kwargs.update(input_params_kwargs2)
        params = sprit.input_params(**input_params_kwargs)

        get_metadata_kwargs = {k: v for k, v in locals()['readcsv_getMeta_fetch_kwargs'].items() if k in sprit.get_metadata.__code__.co_varnames}
        get_metadata_kwargs2 = {k: v for k, v in param_dict.items() if k in sprit.get_metadata.__code__.co_varnames}
        get_metadata_kwargs.update(get_metadata_kwargs2)
        params = sprit.get_metadata(params=params, **get_metadata_kwargs)

        print(params.keys())
        fetch_data_kwargs = {k: v for k, v in locals()['readcsv_getMeta_fetch_kwargs'].items() if k in sprit.fetch_data.__code__.co_varnames}
        fetch_data_kwargs2 = {k: v for k, v in param_dict.items() if k in sprit.fetch_data.__code__.co_varnames[0:7]}
        fetch_data_kwargs.update(fetch_data_kwargs2)
        params = sprit.fetch_data(params=params, **fetch_data_kwargs)
        print(params)

        #if isinstance(read_params, dict):
        #    stream = obspy.read(file, **read_params)
        #else:
        #    stream = obspy.read(file)

        # Get the network, station, and location codes
        network = stream[0].stats.network
        station = stream[0].stats.station
        location = stream[0].stats.location
        
        # Create a unique identifier for the network-station-location combination
        stream_id = f"{network}.{station}.{location}"

        # Check if the stream has a single trace
        if len(stream) == 1:

            # Check if the stream ID exists in the dictionary
            if stream_id in stream_dict:
                if stream[0].stats.channel == 'Z':
                    stream_dict[stream_id][0] = stream[0]
                elif stream[0].stats.channel == 'E':
                    stream_dict[stream_id][1] = stream[0]
                elif stream[0].stats.channel == 'N':
                    stream_dict[stream_id][2] = stream[0]
            else:
                # Create a new stream object with the first trace and assign Z, E, and N channels
                new_stream = obspy.Stream(traces=[None, None, None])
                if stream[0].stats.channel == 'Z':
                    new_stream[0] = stream[0]
                elif stream[0].stats.channel == 'E':
                    new_stream[1] = stream[0]
                elif stream[0].stats.channel == 'N':
                    new_stream[2] = stream[0]
                # Assign network, station, location, etc. to the new_stream as required
                stream_dict[stream_id] = new_stream
        else:
            # Check if the stream has required channels and add it directly to stream_dict
            if has_required_channels(stream):
                stream_dict[stream_id] = stream

    # Validate the channels for each stream in stream_dict
    for stream_id, stream in stream_dict.items():
        if not has_required_channels(stream):
            if verbose:
                print(f"Stream {stream_id} does not have all required channels, removing from analysis.")
            del stream_dict[stream_id]

    if verbose:
        print("Streams:\n")
        print(stream_dict.keys())
        #for k in stream_dict.keys():
        #    print(k)

    return stream_dict

dataPath = r"C:\Users\riley\OneDrive - University of Illinois - Urbana\CodesandScripts\HVSR_Batch_practice.csv"
batch_data_read(input_data=dataPath, batch_type='csv', param_col=None, batch_params=None, verbose=False)

   ID                                           datapath     site  xcoord   
0   0  \\isgs-sinkhole.ad.uillinois.edu\geophysics\HV...  BOM5_23       0  \
1   1  \\isgs-sinkhole.ad.uillinois.edu\geophysics\HV...     BNE4       0   
2   2  \\isgs-sinkhole.ad.uillinois.edu\geophysics\HV...   BOP2_1  352932   

    ycoord  elevation   acq_date starttime endtime station       tzone source   
0        0          0  7/25/2023     10:18   10:43   RAC84  US/Central   file  \
1        0          0  7/10/2023     22:37   22:59   RAC99         UTC   file   
2  4656098        727  7/12/2023     10:41   11:03   RAC84  US/Central    raw   

                          trim_dir  
0                              NaN  
1                              NaN  
2  C:\Users\riley\OneDrive\Desktop  
[{'datapath': '\\\\isgs-sinkhole.ad.uillinois.edu\\geophysics\\HVSR\\BooneCo\\BooneCo23\\HVSRData\\Raw\\BOM5_23_AM.RAC84.00.2023.206_2023-07-25_1518-1543.MSEED', 'site': 'BOM5_23', 'xcoord': 0, 'ycoord': 0, 'elevation'

FileNotFoundError: [Errno 2] No such file or directory: 'nan/BOM5_23_AM.RAC84.00.2023.206_2023-07-25_1518-1542.mseed'